In [1]:
import os
import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, Vector
from dotenv import load_dotenv
import json

import sys
sys.path.append('../app/backend')
from core.modelhelper import get_token_limit
from core.messagebuilder import MessageBuilder
from text import nonewlines

with open('../.azure/config.json', 'r') as config_file:
    config_data = json.load(config_file)
dotenv_path = f'../.azure/{config_data["defaultEnvironment"]}/.env'
load_dotenv(dotenv_path) # Load environment variables from .env file

# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT") 
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER") 
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") 
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") 
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") 
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_MODEL = os.environ.get("AZURE_OPENAI_CHATGPT_MODEL")

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") 
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") 
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE")

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service

# Used by the OpenAI SDK
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"

"# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = \"azure\" instead\n"
#openai.api_type = "azure_ad"
openai.api_key = os.environ.get("OPENAI_API_KEY")#azure_credential.get_token("https://cognitiveservices.azure.com/.default").token

# Set up clients for Cognitive Search and Storage
admin_key = os.environ.get("AZURE_COGNITIVE_SEARCH_KEY")

search_client = SearchClient(
    endpoint= os.environ.get("AZURE_COGNITIVE_SEARCH_ENDPOINT"),
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(admin_key))

In [2]:
from azure.storage.blob import BlobServiceClient
blob_client = BlobServiceClient(
    account_url=f"https://{AZURE_STORAGE_ACCOUNT}.blob.core.windows.net", 
    credential=os.environ.get("AZURE_SAS_TOKEN"))
blob_container = blob_client.get_container_client(AZURE_STORAGE_CONTAINER)

In [3]:
SYSTEM = "system"
USER = "user"
ASSISTANT = "assistant"

system_message_chat_conversation = """You are a customer service assistant for BSH company, helping customers with their home appliance questions, including inquiries about purchasing new products, features, configurations, and troubleshooting.
Start answering thanking the user for their question. Respond in a slightly informal, and helpful tone, with a brief and clear answers. 
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know without referring to the sources. 
Do not generate answers that don't use the sources below and avoid to just cite the source without answering the question. 
If asking a clarifying question to the user would help, ask the question. 
For tabular information, return it as an HTML table. Do not return markdown format. 
If the question is not in English, answer in the language used in the question. 
Each source has a name followed by a colon and the actual information; always include the source name for each fact you use in the response without referring to the sources. 
For example, if the question is 'What is the capacity of this washing machine?' and one of the information sources says 'WGB256090_EN-54.pdf: the capacity is 5kg', then answer with 'The capacity is 5kg [WGB256090_EN-54.pdf]'. 
If there are multiple sources, cite each one in their own square brackets. For example, use '[WGB256090_EN-54.pdf][SMS8YCI03E_EN-24.pdf]' and not in '[WGB256090_EN-54.pdf, SMS8YCI03E_EN-24.pdf]'. 
The name of the source follows a special format: <model_number>_<document_language>-<page_number>.pdf. 
You can Use this information from source name, especially if someone is asking a question about a specific model.
{follow_up_questions_prompt}
{injected_prompt}
"""
follow_up_questions_prompt_content = """Generate three very brief follow-up questions that the user would likely ask next about the home appliance they are interested in or need help with. 
Use double angle brackets to reference the questions, e.g. <<Is there a warranty on this washing machine?>>. 
Try not to repeat questions that have already been asked. 
Only generate questions and do not generate any text before or after the questions, such as 'Next Questions'
"""

query_prompt_template = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base about BSH company's home appliances, including buying guides, features, configurations, and troubleshooting.
Generate a search query based on the conversation and the new question. 
Do not include cited source filenames and document names e.g info.txt or doc.pdf in the search query terms.
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
If the question is not in English, translate the question to English before generating the search query.
If you cannot generate a search query, return just the number 0.
"""
query_prompt_few_shots = [
{'role' : USER, 'content' : 'how to load the washing machine?' },
{'role' : ASSISTANT, 'content' : 'Show the procedure to load a washing machine' },
{'role' : USER, 'content' : 'Does my washing machine has wifi?' },
{'role' : ASSISTANT, 'content' : 'Check for the wifi feature on the specified washing machine' }

]

filter_prompt_template = """Below is a history of the conversation so far, and a new question asked by the user. 
First step: identify the language of the question and return "en-us" if it's in english and "de-de" if it's in german.
If you don't know the language, return "unknown".
Possible aswers are: "en-us", "de-de", "unknown".
Second step: identify the product mentioned in the question and return the product id.
If you don't know the which product the client is talking about because it's not mentioned explicitly in the question, return "unknown".
Product ids are only "SMS6TCI00E", "WUU28TA8", if it's not one of these two, return "unknown".
Possible answers are: "SMS6TCI00E", "WUU28TA8", "unknown".

Return the two answers separated by a comma, e.g. "en-us,SMS6TCI00E".
"""

filter_prompt_few_shots = [
{'role' : USER, 'content' : 'how to load the washing machine?' },
{'role' : ASSISTANT, 'content' : 'en-us,unknown' }, 
{'role' : USER, 'content' : 'what are the available programms for SMS6TCI00E whashing machine?' },
{'role' : ASSISTANT, 'content' : 'en-us,SMS6TCI00E' }, 
{'role' : USER, 'content' : 'what are the available programms for the whashing machine?' },
{'role' : ASSISTANT, 'content' : 'en-us,unknown' }
]


In [4]:
request = {'history': [{'user': 'There is too much vibrations on my washer, how can I solve the problem?'}],
  'approach': 'rrr', 'overrides': {'retrieval_mode': 'hybrid', 'semantic_ranker': True, 'semantic_captions': False, 'top': 5, 'suggest_followup_questions': False}}
overrides = request['overrides']
history = request["history"]

chatgpt_token_limit = get_token_limit(AZURE_OPENAI_CHATGPT_MODEL)

def get_messages_from_history(system_prompt: str, model_id: str, history, user_conv: str, few_shots = [], max_tokens: int = 4096):
    message_builder = MessageBuilder(system_prompt, model_id)

    # Add examples to show the chat what responses we want. It will try to mimic any responses and make sure they match the rules laid out in the system message.
    for shot in few_shots:
        message_builder.append_message(shot.get('role'), shot.get('content'))

    user_content = user_conv
    append_index = len(few_shots) + 1

    message_builder.append_message(USER, user_content, index=append_index)

    for h in reversed(history[:-1]):
        if h.get("bot"):
            message_builder.append_message(ASSISTANT, h.get('bot'), index=append_index)
        message_builder.append_message(USER, h.get('user'), index=append_index)
        if message_builder.token_length > max_tokens:
            break
    
    messages = message_builder.messages
    return messages

In [17]:
has_text = overrides.get("retrieval_mode") in ["text", "hybrid", None]
has_vector = overrides.get("retrieval_mode") in ["vectors", "hybrid", None]
use_semantic_captions = True if overrides.get("semantic_captions") and has_text else False
top = overrides.get("top") or 3
exclude_category = overrides.get("exclude_category") or None
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

user_q = 'User question: ' + history[-1]["user"]

print("prompt for query generation: " + user_q + "\n")

# STEP 1: Generate an optimized keyword search query based on the chat history and the last question
messages_filtering = get_messages_from_history(
    filter_prompt_template,
    AZURE_OPENAI_CHATGPT_MODEL,
    history,
    user_q,
    filter_prompt_few_shots,
    chatgpt_token_limit - len(user_q)
    )

print("Message from chat history: " + str(messages_filtering) + "\n")

chat_completion_filter = await openai.ChatCompletion.acreate(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=messages_filtering,
    temperature=0.0,
    max_tokens=32,
    n=1)

filtering_content = chat_completion_filter.choices[0].message.content
print(filtering_content)

prompt for query generation: User question: There is too much vibrations on my washer, how can I solve the problem?

Message from chat history: [{'role': 'system', 'content': 'Below is a history of the conversation so far, and a new question asked by the user. \nFirst step: identify the language of the question and return "en-us" if it\'s in english and "de-de" if it\'s in german.\nIf you don\'t know the language, return "unknown".\nPossible aswers are: "en-us", "de-de", "unknown".\nSecond step: identify the product mentioned in the question and return the product id.\nIf you don\'t know the which product the client is talking about because it\'s not mentioned explicitly in the question, return "unknown".\nProduct ids are only "SMS6TCI00E", "WUU28TA8", if it\'s not one of these two, return "unknown".\nPossible answers are: "SMS6TCI00E", "WUU28TA8", "unknown".\n\nReturn the two answers separated by a comma, e.g. "en-us,SMS6TCI00E".\n'}, {'role': 'assistant', 'content': 'en-us,unknown'},

In [18]:
user_q = 'Generate search query for: ' + history[-1]["user"]
print("prompt for query generation: " + user_q + "\n")

# STEP 1: Generate an optimized keyword search query based on the chat history and the last question
messages = get_messages_from_history(
    query_prompt_template,
    AZURE_OPENAI_CHATGPT_MODEL,
    history,
    user_q,
    query_prompt_few_shots,
    chatgpt_token_limit - len(user_q)
    )

print("Message from chat history: " + str(messages) + "\n")

chat_completion = await openai.ChatCompletion.acreate(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=messages,
    temperature=0.0,
    max_tokens=32,
    n=1)

response_content = chat_completion.choices[0].message.content
print(response_content)

prompt for query generation: Generate search query for: There is too much vibrations on my washer, how can I solve the problem?

Message from chat history: [{'role': 'system', 'content': "Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base about BSH company's home appliances, including buying guides, features, configurations, and troubleshooting.\nGenerate a search query based on the conversation and the new question. \nDo not include cited source filenames and document names e.g info.txt or doc.pdf in the search query terms.\nDo not include any text inside [] or <<>> in the search query terms.\nDo not include any special characters like '+'.\nIf the question is not in English, translate the question to English before generating the search query.\nIf you cannot generate a search query, return just the number 0.\n"}, {'role': 'assistant', 'content': 'Check for the wifi feature on the specified was

In [19]:
language_code_with_country, product_query = filtering_content.split(",")
# Constructing the language filter
if language_code_with_country in ["en-us", "de-de"]:
    language_filter = f"language eq '{language_code_with_country}'"
    if product_query != "unknown":
        product_filter = f"product_id eq '{product_query}'"
        language_filter = f"{language_filter} and {product_filter}"
else:
    language_filter = ""
    if product_query != "unknown":
        language_filter = f"product_id eq '{product_query}'"
print("Filter: " + language_filter + "\n")

Filter: language eq 'en-us' and product_id eq 'WUU28TA8'



In [20]:
response_content

'"Reducing vibrations on BSH washing machine"'

In [21]:
has_vector

True

In [25]:
# If retrieval mode includes vectors, compute an embedding for the query
if has_vector:
    query_vector = (await openai.Embedding.acreate(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=response_content))["data"][0]["embedding"]
else:
    query_vector = None

    # Only keep the text query if the retrieval mode uses text, otherwise drop it
if not has_text:
    response_content = None

# Use semantic L2 reranker if requested and if retrieval mode is text or hybrid (vectors + text)
if overrides.get("semantic_ranker") and has_text:
    r = search_client.search(response_content,
                                    filter=language_filter,
                                    query_type=QueryType.SEMANTIC,
                                    query_language=language_code_with_country,
                                    query_speller="lexicon",
                                    semantic_configuration_name="default",
                                    top=top,
                                    query_caption="extractive|highlight-false" if use_semantic_captions else None,
                                    vector=query_vector,
                                    top_k=50 if query_vector else None,
                                    vector_fields="embedding" if query_vector else None)
else:
    r = search_client.search(response_content,
                                    filter=language_filter,
                                    top=top,
                                    vector=query_vector,
                                    top_k=50 if query_vector else None,
                                    vector_fields="embedding" if query_vector else None)

In [26]:
response_content

'"Reducing vibrations on BSH washing machine"'

In [27]:
search_client

<SearchClient [endpoint='https://cgs-learning-silvia-basic.search.windows.net', index='ixbshopenai']>

In [28]:
if use_semantic_captions:
    results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + nonewlines(" . ".join([c.text for c in doc['@search.captions']])) for doc in r]
else:
    results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + nonewlines(doc[KB_FIELDS_CONTENT]) for doc in r]
content = "\n".join(results)

In [29]:
print(content)

WUU28TA8_en-us_washing machine-14.pdf:  ▶Secure the water outlet hose against unintentional loosening. Note:  Observe the draining heights. The maximum draining height is 100 cm.Siphon  Secure the con- nection point with a (24–40 mm) hose clamp. Sink  Fix and secure the water outlet hose with an el- bow → Page  28. Plastic pipe with rubber sleeve or gully Fix and secure the water outlet hose with an el- bow → Page  28. 4.6 Aligning the appliance In order to reduce noise and vibra- tion and to prevent the appliance from moving, align the appliance cor- rectly. 1.Loosen the lock nuts using a WAF 17 wrench by turning it in a clock- wise direction. en Installation and connection 162.To align the appliance, turn the ap- pliance feet. Check that the appli- ance is level using a spirit level. All appliance feet must stand firmly on the ground. 3.Screw the lock nuts tightly against the housing using a WAF 17 wrench. In doing so, hold the foot tight and do not adjust its height. 4.7 Connecting 

In [30]:
print("Retrieved documents: " + content + "\n")

follow_up_questions_prompt = follow_up_questions_prompt_content if overrides.get("suggest_followup_questions") else ""

print("Follow up questions prompt: " + follow_up_questions_prompt + "\n")

# STEP 3: Generate a contextual and content specific answer using the search results and chat history

# Allow client to replace the entire prompt, or to inject into the exiting prompt using >>>
prompt_override = overrides.get("prompt_override")
if prompt_override is None:
    system_message = system_message_chat_conversation.format(injected_prompt="", follow_up_questions_prompt=follow_up_questions_prompt)
elif prompt_override.startswith(">>>"):
    system_message = system_message_chat_conversation.format(injected_prompt=prompt_override[3:] + "\n", follow_up_questions_prompt=follow_up_questions_prompt)
else:
    system_message = prompt_override.format(follow_up_questions_prompt=follow_up_questions_prompt)

messages = get_messages_from_history(
    system_message + "\n\nSources:\n" + content,
    AZURE_OPENAI_CHATGPT_MODEL,
    history,
    history[-1]["user"],
    max_tokens=chatgpt_token_limit)

print("Message from chat history: " + str(messages) + "\n")

chat_completion = await openai.ChatCompletion.acreate(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=messages,
    temperature=overrides.get("temperature") or 0.7,
    max_tokens=1024,
    n=1)

print("Generated answer: " + chat_completion.choices[0].message.content + "\n")

chat_content = chat_completion.choices[0].message.content

print("Chat content: " + chat_content + "\n")

msg_to_display = '\n\n'.join([str(message) for message in messages])

print("Message to display: " + msg_to_display + "\n")

Retrieved documents: WUU28TA8_en-us_washing machine-14.pdf:  ▶Secure the water outlet hose against unintentional loosening. Note:  Observe the draining heights. The maximum draining height is 100 cm.Siphon  Secure the con- nection point with a (24–40 mm) hose clamp. Sink  Fix and secure the water outlet hose with an el- bow → Page  28. Plastic pipe with rubber sleeve or gully Fix and secure the water outlet hose with an el- bow → Page  28. 4.6 Aligning the appliance In order to reduce noise and vibra- tion and to prevent the appliance from moving, align the appliance cor- rectly. 1.Loosen the lock nuts using a WAF 17 wrench by turning it in a clock- wise direction. en Installation and connection 162.To align the appliance, turn the ap- pliance feet. Check that the appli- ance is level using a spirit level. All appliance feet must stand firmly on the ground. 3.Screw the lock nuts tightly against the housing using a WAF 17 wrench. In doing so, hold the foot tight and do not adjust its he

In [32]:
print(chat_content)

To reduce vibrations and prevent the appliance from moving, you can align the appliance correctly. First, loosen the lock nuts using a WAF 17 wrench by turning it in a clockwise direction. Then, turn the appliance feet to align it and check that all appliance feet stand firmly on the ground using a spirit level. Finally, screw the lock nuts tightly against the housing using a WAF 17 wrench. Remember to hold the foot tight and not adjust its height while doing so. [WUU28TA8_en-us_washing machine-14.pdf]


In [3]:
import sys 
sys.path.append('../app/backend')
from approaches.chatreadretrieveread import ChatReadRetrieveReadApproach

In [4]:
chat = ChatReadRetrieveReadApproach(search_client, 
                                        AZURE_OPENAI_CHATGPT_DEPLOYMENT,
                                        AZURE_OPENAI_CHATGPT_MODEL, 
                                        AZURE_OPENAI_EMB_DEPLOYMENT,
                                        KB_FIELDS_SOURCEPAGE, 
                                        KB_FIELDS_CONTENT)

In [5]:
request = {'history': [{'user': 'Could you provide some insights on what I can consider for asset tracking?'}],
  'approach': 'rrr', 'overrides': 
  {'retrieval_mode': 'hybrid', 'semantic_ranker': True, 'semantic_captions': False, 'top': 5, 'suggest_followup_questions': False}}

In [6]:
overrides = request["overrides"]
history = request["history"]

In [7]:
r = chat.run(request["history"], request.get("overrides") or {})

In [8]:
print(r["answer"])

TypeError: 'coroutine' object is not subscriptable